### Package installation

In [23]:
# pip install psycopg2
# pip install pandas
# pip install openpyxl

### Package call

In [46]:
import psycopg2
import pandas as pd
import psycopg2.extras

### Connect to the database

In [47]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "123456",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection

In [69]:
con = PostgresConnection().getConnection()

successfully connected to database


### Creating Table

In [55]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
        """ CREATE TABLE ecomdb_star_schema.trans_dim
            (
             payment_key  varchar(10) NOT NULL,
             trans_type varchar(10) NOT NULL,
             bank_name  text NOT NULL,
             CONSTRAINT PK_10 PRIMARY KEY ( payment_key )
            )
        """,
        
        """ CREATE TABLE ecomdb_star_schema.coustomer_dim
            (
             coustomer_key varchar(10) NOT NULL,
             name          varchar(20) NOT NULL,
             contact_no    bigint NOT NULL,
             nid           bigint NOT NULL,
             CONSTRAINT PK_62 PRIMARY KEY ( coustomer_key )
            )
        """
        ,
        
        """
         CREATE TABLE Time_dim(
             time_key varchar(10) NOT NULL,
             "date"     timestamp NOT NULL,
             day      int NOT NULL,
             hour     int NOT NULL,
             week     varchar(10) NOT NULL,
             month    int NOT NULL,
             quarter  varchar(5) NOT NULL,
             year     int NOT NULL,
             CONSTRAINT PK_77 PRIMARY KEY ( time_key )
            )
        """
        ,
        
        """
         CREATE TABLE item_dim(
             item_key    varchar(50) NOT NULL,
             item_name   text NOT NULL,
             "desc"        text NOT NULL,
             unit_price  numeric NOT NULL,
             man_country varchar(50) NOT NULL,
             supplier    varchar(50) NOT NULL,
             unit        varchar(10) NOT NULL,
             CONSTRAINT PK_35 PRIMARY KEY ( item_key )
            )
        
        """
        ,
        
        """ CREATE TABLE ecomdb_star_schema.store_dim
            (
             store_key varchar(10) NOT NULL,
             division  varchar(50) NOT NULL,
             district  varchar(50) NOT NULL,
             upazila   varchar(50) NOT NULL,
             CONSTRAINT PK_71 PRIMARY KEY ( store_key )
            )
            
        """
        ,
        
        """  CREATE TABLE ecomdb_star_schema.fact_table
                (
                 "id"              int NOT NULL,
                 payment_key     varchar(30) NOT NULL,
                 time_key_1      varchar(10) NOT NULL,
                 store_key_1     varchar(10) NOT NULL,
                 payment_key_1   varchar(10) NOT NULL,
                 coustomer_key_1 varchar(10) NOT NULL,
                 coustomer_key   varchar(30) NOT NULL,
                 time_key        varchar(30) NOT NULL,
                 item_key        varchar(30) NOT NULL,
                 store_key       varchar(30) NOT NULL,
                 quantity        integer NOT NULL,
                 unit            varchar(10) NOT NULL,
                 unit_price      decimal(18,0) NOT NULL,
                 total_price     decimal(18,0) NOT NULL,
                 item_key_1      varchar(50) NOT NULL,
                 CONSTRAINT PK_19 PRIMARY KEY ( "id" ),
                 CONSTRAINT FK_108 FOREIGN KEY ( time_key_1 ) REFERENCES ecomdb_star_schema.time_dim ( time_key ),
                 CONSTRAINT FK_33 FOREIGN KEY ( item_key_1 ) REFERENCES ecomdb_star_schema.item_dim ( item_key ),
                 CONSTRAINT FK_86 FOREIGN KEY ( coustomer_key_1 ) REFERENCES ecomdb_star_schema.coustomer_dim ( coustomer_key ),
                 CONSTRAINT FK_89 FOREIGN KEY ( payment_key_1 ) REFERENCES ecomdb_star_schema.trans_dim ( payment_key ),
                 CONSTRAINT FK_92 FOREIGN KEY ( store_key_1 ) REFERENCES ecomdb_star_schema.store_dim ( store_key )
                )

        """
        ,
        
        """
        CREATE INDEX FK_91 ON ecomdb_star_schema.fact_table
            (
             payment_key_1
            )
        """
        ,
        
        """ CREATE INDEX FK_88 ON ecomdb_star_schema.fact_table
            (
             coustomer_key_1
            )
        """
        ,
        
        """ CREATE INDEX FK_110 ON ecomdb_star_schema.fact_table
            (
             time_key_1
            )
        """ 
        ,
        
        """ CREATE INDEX FK_36 ON ecomdb_star_schema.fact_table
            (
             item_key_1
            )
        """
        ,
        
        """
        CREATE INDEX FK_94 ON ecomdb_star_schema.fact_table
            (
             store_key_1
            )
        """
        )
    try:
        cur = con.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        cur.close() # close communication with the PostgreSQL database server
        con.commit()# commit the changes
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if con is not None:
            con.close()


if __name__ == '__main__':
 

   create_tables()



SyntaxError: invalid syntax (182322120.py, line 23)

### Extract
***Read from excel file***

In [3]:
# DATA_DIR = "/home/raihan/eSRD-Lab/HDA/dataset/"
DATA_DIR = "D:/eSRD_Lab/DataBase/"

In [31]:
# can ignore engine name . 
# can use name text instead of number. 
# Can ignore read mode also as it is byv default
# trans_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name=1, engine='openpyxl')

#### Extracting Fact_table to data frame  and data preprocessing

In [4]:
df_fact_table = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name="Fact_table")
df_fact_table

df_fact_table.isna()
df_fact_table.isna().sum()
# male = df['Sex'][df['Sex'] == 'male'].count()
# male

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0
1,P022,C008967,T041209,I00248,S00595,1,rolls,26.0,26.0
2,P030,C007261,T03633,I00195,S00496,8,ct,12.5,100.0
3,P032,C007048,T084631,I00131,S0086,8,ct,14.0,112.0
4,P014,C006430,T071276,I00050,S00488,8,cans,8.0,64.0
...,...,...,...,...,...,...,...,...,...
999995,P010,C003726,T09247,I00063,S00502,5,cans,40.0,200.0
999996,P021,C002581,T023483,I00252,S00225,1,ct,11.0,11.0
999997,P035,C006133,T039372,I00119,S00555,7,ct,53.0,371.0
999998,P031,C008735,T097975,I00257,S00440,2,ct,21.0,42.0


In [18]:
df_fact_table.isna().sum()

payment_key         0
coustomer_key       0
time_key            0
item_key            0
store_key           0
quantity            0
unit             3723
unit_price          0
total_price         0
dtype: int64

In [27]:
df_fact_table['unit'].unique()

array(['ct', 'rolls', 'cans', 'bottles', 'bags', 'oz', 'tubs', 'cartons',
       'lb', 'tins', 'ct.', 'pack', 'pk', 'bars', 'Bags', 'oz.', nan,
       'Ct', 'botlltes'], dtype=object)

In [28]:
df_fact_table["unit"].fillna("ct", inplace = True)
df_fact_table.isna().sum()

payment_key      0
coustomer_key    0
time_key         0
item_key         0
store_key        0
quantity         0
unit             0
unit_price       0
total_price      0
dtype: int64

### Extracting trans_dim to data frame  and data Preprocessing 

In [5]:
df_trans_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name="Trans_dim")
df_trans_dim

,payment_key,trans_type,bank_name
0,P001,cash,None
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited


***Insert and check trans dim data***

In [30]:
insert_stmt = "INSERT INTO ecomdb_star_schema.trans_dim ( payment_key, trans_type, bank_name)" \
            " VALUES (%s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, df_trans_dim.values)
con.commit()
cur.close()

In [15]:
cur = con.cursor()
select_stmt = "SELECT t.payment_key , t.trans_type, t.bank_name " \
              "FROM ecomdb_star_schema.trans_dim t"
cur.execute(select_stmt)
records = cur.fetchall()
records

[('P001', 'cash', 'None'),
 ('P002', 'card', 'AB Bank Limited'),
 ('P003', 'card', 'Bangladesh Commerce Bank Limited'),
 ('P004', 'card', 'Bank Asia Limited'),
 ('P005', 'card', 'BRAC Bank Limited'),
 ('P006', 'card', 'Citizens Bank Limited'),
 ('P007', 'card', 'City Bank Limited'),
 ('P008', 'card', 'Community Bank Bangladesh Limited'),
 ('P009', 'card', 'Dhaka Bank Limited'),
 ('P010', 'card', 'Dutch-Bangla Bank Limited'),
 ('P011', 'card', 'Eastern Bank Limited'),
 ('P012', 'card', 'IFIC Bank Limited'),
 ('P013', 'card', 'Jamuna Bank Limited'),
 ('P014', 'card', 'Meghna Bank Limited'),
 ('P015', 'card', 'Mercantile Bank Limited'),
 ('P016', 'card', 'Midland Bank Limited'),
 ('P017', 'card', 'Modhumoti Bank Limited'),
 ('P018', 'card', 'Mutual Trust Bank Limited'),
 ('P019', 'card', 'National Bank Limited'),
 ('P020', 'card', 'National Credit & Commerce Bank Limited'),
 ('P021', 'card', 'NRB Bank Limited'),
 ('P022', 'card', 'NRB Commercial Bank Ltd'),
 ('P023', 'card', 'NRB Global B

In [16]:
trans_df = pd.DataFrame(list(records), columns=['payment_key', 'trans_type', 'bank_name'])
trans_df

,payment_key,trans_type,bank_name
0,P001,cash,None
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited


### Extracting item_dim to data frame  and data Preprocessing 

In [62]:
df_item_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name="Item_dim")
df_item_dim = df_item_dim.rename(columns = {'desc':'description'}, inplace = False)
df_item_dim

,item_key,item_name,description,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a. Beverage - Soda,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a. Beverage - Soda,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a. Beverage - Soda,6.75,Finland,HARDFORD AB,cans
...,...,...,...,...,...,...,...
259,I00260,Napkins Square Lunch,Kitchen Supplies,15.00,Cambodia,NINGBO SEDUNO IMP & EXP CO.LTD,ct
260,I00261,POM 2 ply paper towels,Kitchen Supplies,33.00,Lithuania,BIGSO AB,ct
261,I00262,Strong Everyday Napkins,Kitchen Supplies,14.00,Netherlands,Bolsius Boxmeer,ct
262,I00263,Advil 2 pill packets,Medicine,14.00,Lithuania,BIGSO AB,Ct


In [63]:
df_item_dim.isna().sum()

item_key       0
item_name      0
description    0
unit_price     0
man_country    0
supplier       0
unit           1
dtype: int64

In [64]:
# df_item_dim.loc[(df_item_dim.unit == null),'unit']='ct'
df_item_dim["unit"].fillna("ct", inplace = True)
df_item_dim.isna().sum()

item_key       0
item_name      0
description    0
unit_price     0
man_country    0
supplier       0
unit           0
dtype: int64

In [26]:
df_item_dim['supplier'].unique()

array(['Bolsius Boxmeer', 'CHROMADURLIN S.A.S', 'DENIMACH LTD',
       'HARDFORD AB', 'BIGSO AB', 'Indo Count Industries Ltd',
       'Friedola 1888 GmbH', 'CHERRY GROUP CO.,LTD', 'MAESA SAS',
       'NINGBO SEDUNO IMP & EXP CO.LTD'], dtype=object)

In [70]:
def insert_table_data(insert_stmt, df):
    try:
        cur = con.cursor()
        psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
        con.commit()
        cur.close() # close communication with the PostgreSQL database server
       
    except (Exception, psycopg2.DatabaseError) as error:
        con.rollback()
        print(error)
    finally:
        if con is not None:
            con.close()



In [71]:
insert_stmt = "INSERT INTO ecomdb_star_schema.item_dim ( item_key, item_name, description, unit_price, man_country, supplier, unit)" \
            " VALUES (%s, %s, %s, %s, %s, %s, %s)"
insert_table_data(insert_stmt,df_item_dim)

# Extracting coustomer_dim to data frame  and data Cleaning 

In [7]:
df_coustomer_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name="Coustomer_dim")
df_coustomer_dim

,coustomer_key,name,contact_no,nid
0,C000001,sumit,8801920345851,7505075708899
1,C000002,tammanne,8801817069329,1977731324842
2,C000003,kailash kumar,8801663795774,3769494056318
3,C000004,bhagwati prasad,8801533627961,9378834712725
4,C000005,ajay,8801943715786,3540815556323
...,...,...,...,...
9186,C009187,smt radha,8801563029753,1057824928189
9187,C009188,tulsi sharma,8801794982266,7970880905205
9188,C009189,madhubala,8801890032435,3101930298150
9189,C009190,pintu devi,8801563290767,7234328323528


In [8]:
df_time_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name="Time_dim")
df_time_dim

,time_key,date,hour,day,week,month,quarter,year
0,T00001,20-05-2017 14:56,14,20,3rd Week,5,Q2,2017
1,T00002,30-01-2015 22:14,22,30,4th Week,1,Q1,2015
2,T00003,14-03-2020 02:34,2,14,2nd Week,3,Q1,2020
3,T00004,27-04-2018 12:19,12,27,4th Week,4,Q2,2018
4,T00005,14-04-2018 10:43,10,14,2nd Week,4,Q2,2018
...,...,...,...,...,...,...,...,...
99994,T099995,04-01-2015 17:22,17,4,1st Week,1,Q1,2015
99995,T099996,17-01-2015 21:09,21,17,3rd Week,1,Q1,2015
99996,T099997,23-01-2020 14:42,14,23,4th Week,1,Q1,2020
99997,T099998,31-10-2015 13:52,13,31,4th Week,10,Q4,2015


# Extracting store_dim to data frame  and data preprocessing 

In [9]:
df_store_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name="Store_dim")
df_store_dim

,store_key,division,district,upazila
0,S0001,SYLHET,HABIGANJ,AJMIRIGANJ
1,S0002,SYLHET,HABIGANJ,BAHUBAL
2,S0003,SYLHET,HABIGANJ,BANIACHONG
3,S0004,SYLHET,HABIGANJ,CHUNARUGHAT
4,S0005,SYLHET,HABIGANJ,HABIGANJ SADAR
...,...,...,...,...
721,S00722,BARISAL,BARISAL,HIZLA
722,S00723,BARISAL,BARISAL,BARISAL SADAR (KOTWALI)
723,S00724,BARISAL,BARISAL,MEHENDIGANJ
724,S00725,BARISAL,BARISAL,MULADI


In [35]:
# df_store_dim["unit"].fillna("ct", inplace = True)
df_store_dim.isna().sum()

store_key    0
division     0
district     0
upazila      0
dtype: int64

#insert and check store data

In [43]:
insert_stmt = "INSERT INTO ecomdb_star_schema.store_dim (store_key, division, district,upazila)" \
            " VALUES (%s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, df_store_dim.values)
con.commit()
cur.close()

In [44]:
cur = con.cursor()
select_stmt = "SELECT * FROM ecomdb_star_schema.store_dim"
cur.execute(select_stmt)
records = cur.fetchall()
records

[('S0001', 'SYLHET', 'HABIGANJ', 'AJMIRIGANJ'),
 ('S0002', 'SYLHET', 'HABIGANJ', 'BAHUBAL'),
 ('S0003', 'SYLHET', 'HABIGANJ', 'BANIACHONG'),
 ('S0004', 'SYLHET', 'HABIGANJ', 'CHUNARUGHAT'),
 ('S0005', 'SYLHET', 'HABIGANJ', 'HABIGANJ SADAR'),
 ('S0006', 'SYLHET', 'HABIGANJ', 'LAKHAI'),
 ('S0007', 'SYLHET', 'HABIGANJ', 'MADHABPUR'),
 ('S0008', 'SYLHET', 'HABIGANJ', 'NABIGANJ'),
 ('S0009', 'SYLHET', 'MAULVIBAZAR', 'BARLEKHA'),
 ('S0010', 'SYLHET', 'MAULVIBAZAR', 'JURI'),
 ('S0011', 'SYLHET', 'MAULVIBAZAR', 'KAMALGANJ'),
 ('S0012', 'SYLHET', 'MAULVIBAZAR', 'KULAURA'),
 ('S0013', 'SYLHET', 'MAULVIBAZAR', 'MAULVIBAZAR SADAR'),
 ('S0014', 'SYLHET', 'MAULVIBAZAR', 'RAJNAGAR'),
 ('S0015', 'SYLHET', 'MAULVIBAZAR', 'SREEMANGAL'),
 ('S0016', 'SYLHET', 'SUNAMGANJ', 'BISHWAMBARPUR'),
 ('S0017', 'SYLHET', 'SUNAMGANJ', 'CHHATAK'),
 ('S0018', 'SYLHET', 'SUNAMGANJ', 'DAKSHIN SUNAMGANJ'),
 ('S0019', 'SYLHET', 'SUNAMGANJ', 'DERAI'),
 ('S0020', 'SYLHET', 'SUNAMGANJ', 'DHARAMPASHA'),
 ('S0021', 'SYLHET', 'S

### Transform